In [1]:
import random
from random import shuffle
import pandas as pd

In [3]:
# eine einfache Shuffle-Funktion
def my_shuffle(items):
    """liefert eine permutierte *Kopie* von items zurück"""
    tmp = items[:]      # Kopie
    shuffle(tmp)        # Fisher-Yates
    return tmp

# ------------------------------------
# Hauptfunktion
# ------------------------------------
def partial_shuffle(lst, min_keep_ratio, shuffle_func):
    """
    Behält mindestens `min_keep_ratio`·len(lst) Fixpunkte.
    Die übrigen Positionen werden mit `shuffle_func` neu angeordnet.

    Parameter
    ---------
    lst          : Original-Liste (beliebiger Typ, bleibt unverändert)
    min_keep_ratio : Anteil der Fixpunkte (0 .. 1)
    shuffle_func : Callable, das eine Liste -> permutierte Liste liefert
    """
    if not callable(shuffle_func):
        raise TypeError("shuffle_func must be a callable that returns a shuffled list")

    n = len(lst)
    k = round((1 - min_keep_ratio) * n)      # zu bewegende Elemente
    if k <= 0:
        return lst[:]                        # alles bleibt

    # 1) zufällige Indizes auswählen
    idx     = sorted(random.sample(range(n), k))
    movable = [lst[i] for i in idx]

    # 2) mit der übergebenen Shuffle-Funktion permutieren
    shuffled = shuffle_func(movable)

    # 3) zurückschreiben
    out = lst[:]
    for j, i in enumerate(idx):
        out[i] = shuffled[j]
    return out


In [7]:
# Test
base  = list(range(1, 101))
mixed = partial_shuffle(base, min_keep_ratio=0.8, shuffle_func=my_shuffle)
df = pd.DataFrame({
    'Base': base,
    'Mixed': mixed,
    'Match': [b == m for b, m in zip(base, mixed)]
})
df.head(30)

,Base,Mixed,Match
0,1,1,True
1,2,2,True
2,3,3,True
3,4,18,False
4,5,5,True
5,6,96,False
6,7,7,True
7,8,8,True
8,9,83,False
9,10,10,True


## Block-Cross Shuffle

In [9]:
import random
from collections import deque

def block_cross_shuffle(items, c=3):
    """
    items : Sequenz beliebiger Länge n
    c     : Anzahl möglichst gleich großer, aufeinander­folgender Blöcke (≥2)

    Gibt eine permutierte Kopie von items zurück.
    Jeder Quell-Block tauscht seine Elemente rundenweise mit allen folgenden.
    """

    items = list(items)                    # Kopie
    n     = len(items)
    if not (2 <= c <= n):
        raise ValueError("c must be between 2 and len(items)")

    # 1. Blockgrenzen erzeugen
    cuts   = [round(i * n / c) for i in range(c + 1)]
    blocks = [deque(range(cuts[i], cuts[i + 1])) for i in range(c)]

    # ⇨ Print der tatsächlichen Werte pro Block
    print("Blöcke (Werte):")
    for b in blocks:
        print([items[i] for i in b])
    print()

    # 2. Tauschen
    out = items[:]
    for src in range(c - 1):
        while blocks[src]:
            targets = [t for t in range(src + 1, c) if blocks[t]]
            if not targets:
                break

            t_idx = 0
            while blocks[src] and targets:
                tgt = targets[t_idx]

                i_idx = blocks[src].popleft()
                j_idx = blocks[tgt].popleft()
                out[i_idx], out[j_idx] = out[j_idx], out[i_idx]

                if not blocks[tgt]:
                    targets.pop(t_idx)
                else:
                    t_idx = (t_idx + 1) % len(targets)

    return out

In [11]:
if __name__ == "__main__":
    data  = list(range(1, 30))             # 1 … 24
    mixed = block_cross_shuffle(data, c=5) # 
    print("Vorher :", data)
    print("Nachher:", mixed)


Blöcke (Werte):
[1, 2, 3, 4, 5, 6]
[7, 8, 9, 10, 11, 12]
[13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29]

Vorher : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Nachher: [7, 13, 18, 24, 8, 14, 1, 5, 15, 19, 25, 16, 2, 6, 9, 12, 20, 3, 10, 17, 26, 27, 28, 4, 11, 21, 22, 23, 29]


In [15]:
base  = list(range(1, 101))                
mixed = partial_shuffle(base, min_keep_ratio=0.9,
                        shuffle_func=lambda seq: block_cross_shuffle(seq, c=3)
                       )
df = pd.DataFrame({
    'Base': base,
    'Mixed': mixed,
    'Match': [b == m for b, m in zip(base, mixed)]
})
df.head(20)

Blöcke (Werte):
[5, 6, 9]
[24, 34, 66, 68]
[70, 83, 87]



,Base,Mixed,Match
0,1,1,True
1,2,2,True
2,3,3,True
3,4,4,True
4,5,24,False
5,6,70,False
6,7,7,True
7,8,8,True
8,9,34,False
9,10,10,True
